In [1]:
# AOC - Day 10, part2 - https://adventofcode.com/2019/day/10#part2
# notes on coordinate reference systems
# positions are referenced as a tuple, like (row, col) in the challenge
# if this were an x,y coordinate system - that would actually by (y,x) - a new row is an increase in y
# increasing row counts go 'down' visually, as the data is read in serially
# the challenge specifies that the laser starts by pointing 'up' and then rotates clockwise.
# in conventional cartesian coordinates, given that increasing y is 'down', up would then be
# 180 degrees.  all sorts of accomodations had to be made to get that to work out

In [2]:
# the asteriod class
class Asteriod:
    def __init__(self, x=None, y=None):
        # this is the (x,y) location of this asteroid in cartesian coordinates
        self.loc = (x, y)
        # this is an array for neighbors proximity
        self.neighbors = []

In [3]:
# image map
# not important to the solution, but useful to debug
class Imap:
    def __init__(self, e, x=None, y=None):
        # this is the (x,y) location of this item in cartesian coordinates
        self.loc = (x, y)
        self.entry = e
def showentry(x,y):
    i = [j.entry for j in m if j.loc[0] == x and j.loc[1] == y]
    print("entry at {},{} = {}".format(x,y,i[0]))

In [4]:
# rotate an angle by some value
def rotate(x, d, m):
    # x - the angle to rotate
    # d - how much to rotate
    # m - # of degrees in circle
    n = x + d
    if(n >= m):
        n = n - m
    if(n < 0):
        n = n + m
    return(n)

In [5]:
def proximity(p1, p2):
    import math
    # calculates bearing and distance between 2 points
    # p1, p2 are location tuples of the form: (x,y)
    deltaX = p2[0] - p1[0]
    deltaY = p2[1] - p1[1]
    dist = math.sqrt(deltaX**2 + deltaY**2)
    # calculate bearing, copnvert to degrees
    # why deltaX first?  the gun-angle is given as "up is 0" and referenced then from the y-axis
    # convention for atan2 is referenced from the x-axis
    b = math.atan2(deltaX * -1, deltaY) * (180 / math.pi)
    # need to deal with negative degrees...
    if(b < 0):
        b += 360
    # this accomodates precisely that y increases as you go 'down' visually
    b = rotate(b, 180, 360)
    return(round(dist,6),round(b,6))

In [6]:
# solar system (ss) is an array of Asteroids
m = []
ss = []
# read in the solar system input file as an array of characters ('#', '.', or '\n')
ifn = "day10-input.txt"
d = open(ifn)
s = d.read()

In [7]:
# run through the input array, fill the ss array with asteroids
# initial row, column
row = 0
column = 0
for a in s:
    if(a == '\n'):
        # that's end of the line, increment row, set column to 0
        row += 1
        column = 0
        log_msg = "next...{},{}: {}".format(row,column,a)
    else:
        # check if it is an asteroid
        if(a == '#'):
            # add an asteroid to the ss array, specifying its location in the cartesian system
            # the row is the y, the x is the column postion
            ss.append(Asteriod(x=column,y=row))
        # add to the image map (not important to solution)
        m.append(Imap(a,x=column,y=row))
        # and increment column
        column += 1

In [8]:
# run through each asteroid, calculate and store proximity (distance, bearing) to all the others, as
# well as the loc of that neighbor
for i,p1 in enumerate(ss):
    for j,p2 in enumerate(ss):
        if(i != j):
            p = proximity(p1.loc, p2.loc)
            p1.neighbors.append((p[0], p[1], p2.loc))

In [9]:
set_trace = False
if(set_trace): import pdb; pdb.set_trace()
set_trace = False

In [10]:
# find the one that has the most visible neighbors
# initialize before determining
best_loc = (0,0)
most_visible = 0
a_index = -1
# go through all the asteroids and calculate how many others are visible to each
# and save the location of the one with most
for i, a in enumerate(ss):
    # get the list of bearings for each neighbor, make it a set, return the length of the set
    # unique bearing entries are the number of visible neighbors, and this is what the set() operation does
    # asteroids with the same bearing hide behind each other
    num = len(set([j[1] for j in a.neighbors]))
    if(num > most_visible):
        # if it's the best one, make this the new best, and save the location
        most_visible = num
        best_loc = a.loc
        a_index = i

In [11]:
print("The best location is asteroid #{}, at pos: {}, where {} other asteroids are visible.".format(a_index,best_loc, most_visible))

The best location is asteroid #352, at pos: (17, 23), where 296 other asteroids are visible.


In [12]:
# asteroid kill counter
ast_ctr = 0

In [13]:
# determine which would be the k-th vaporization
k = 199
while(ast_ctr <= k):
    # find the unique set of bearings for all the current neighbors
    o = set([j[1] for j in ss[a_index].neighbors])
    # make a list of the set
    l = list(o)
    # and sort it
    l.sort()
   
    # do a revolution, with one entry per unique bearing
    kill_list = []
    nkill_list = []
    for m,r in enumerate(l):
        if(ast_ctr == k+1):
            break
        else:
            # get all the neighbors at this bearing
            a = [j for j in ss[a_index].neighbors if j[1] == r]
            # print("r: {}, a: {}".format(r, a))
            # find the closest one
            min = 999999
            # go through all of them and find the closest
            for i,b in enumerate(a):
                # if this is the close one...
                if(b[0] < min):
                    # reset min
                    min = b[0]
                    # remember the location
                    loc_to_kill = b[2]
            # remove the asteroid located at loc_to_kill from ss
            kill_list.append(loc_to_kill)
            # print("ast_ctr: {}, bearing: {}, loc_to_kill: {}".format(ast_ctr, r, loc_to_kill))
            # increment the asteroid kill counter
            ast_ctr += 1
            
    # do the kills and the neighbor list maintenance
    for i,kloc in enumerate(kill_list):
        nidx_to_kill = [i for i,e in enumerate(ss[a_index].neighbors) if e[2] == kloc]
        del ss[a_index].neighbors[nidx_to_kill[0]]
        last = kloc

In [14]:
print("The {}th asteroid to be vaporized was located at {}.".format(ast_ctr, last))
print("The number to submit is: {}".format((last[0])*100 + last[1]))

The 200th asteroid to be vaporized was located at (2, 4).
The number to submit is: 204
